# 장기 컨텍스트 기억(Long-term Memory)

이 워크샵에서는 Agent Framework에서 장기 컨텍스트 기억을 구현하는 다양한 방법을 학습합니다.

## 목차
1. [환경 설정](#환경-설정)
2. [간단한 커스텀 Context Provider](#간단한-커스텀-context-provider)
3. [Mem0 Context Provider](#mem0-context-provider)
4. [Redis Context Provider](#redis-context-provider)
5. [Thread 관리 및 스코핑](#thread-관리-및-스코핑)

## 학습 목표
- Context Provider의 개념과 작동 방식 이해
- 커스텀 Context Provider 구현 방법 학습
- Mem0와 Redis를 활용한 영구 메모리 구현
- Thread 스코핑 전략 이해

## 1. 환경 설정

필요한 패키지를 설치하고 환경 변수를 설정합니다.

In [13]:
# 필요한 패키지 설치
# !pip install agent-framework agent-framework-azure-ai agent-framework-mem0 agent-framework-redis python-dotenv

In [1]:
# 환경 변수 로드
import os
from dotenv import load_dotenv

# .env 파일에서 환경 변수 로드 (override=True로 기존 값 덮어쓰기)
load_dotenv(override=True)

# Mem0에 필요한 Azure OpenAI 환경 변수 설정
os.environ["LLM_AZURE_OPENAI_API_KEY"] = os.getenv("AZURE_OPENAI_API_KEY", "")
os.environ["LLM_AZURE_DEPLOYMENT"] = os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "")
os.environ["LLM_AZURE_ENDPOINT"] = os.getenv("AZURE_OPENAI_ENDPOINT", "")
os.environ["LLM_AZURE_API_VERSION"] = "2024-10-21"

# 필요한 환경 변수 확인
required_vars = [
    "AZURE_AI_PROJECT_ENDPOINT",
    "AZURE_AI_MODEL_DEPLOYMENT_NAME",
]

optional_vars = [
    "MEM0_API_KEY",  # Mem0 Platform 사용 시
    "OPENAI_API_KEY",  # Mem0 OSS 또는 Redis 벡터 검색 사용 시
]

print("필수 환경 변수:")
for var in required_vars:
    status = "✓" if os.getenv(var) else "✗"
    print(f"{status} {var}")

print("\n선택 환경 변수:")
for var in optional_vars:
    status = "✓" if os.getenv(var) else "✗"
    print(f"{status} {var}")

print("\nMem0용 Azure OpenAI 환경 변수:")
mem0_vars = [
    "LLM_AZURE_OPENAI_API_KEY",
    "LLM_AZURE_DEPLOYMENT", 
    "LLM_AZURE_ENDPOINT",
    "LLM_AZURE_API_VERSION"
]
for var in mem0_vars:
    value = os.getenv(var, "")
    status = "✓" if value else "✗"
    # API 키는 마스킹해서 표시
    if "API_KEY" in var and value:
        display_value = f"{value[:3]}...{value[-3:]}"
    elif value:
        display_value = value
    else:
        display_value = "Not set"
    print(f"{status} {var}: {display_value}")

필수 환경 변수:
✓ AZURE_AI_PROJECT_ENDPOINT
✓ AZURE_AI_MODEL_DEPLOYMENT_NAME

선택 환경 변수:
✓ MEM0_API_KEY
✗ OPENAI_API_KEY

Mem0용 Azure OpenAI 환경 변수:
✓ LLM_AZURE_OPENAI_API_KEY: 6hB...Teh
✓ LLM_AZURE_DEPLOYMENT: gpt-4.1
✓ LLM_AZURE_ENDPOINT: https://aistudioaiservices343274518006.openai.azure.com
✓ LLM_AZURE_API_VERSION: 2024-10-21


## 2. 간단한 커스텀 Context Provider

먼저 간단한 커스텀 Context Provider를 만들어 기본 개념을 이해해봅시다.
이 예제는 사용자의 이름과 나이를 기억하는 간단한 메모리를 구현합니다.

In [3]:
from collections.abc import MutableSequence, Sequence
from typing import Any

from agent_framework import ChatAgent, ChatClientProtocol, ChatMessage, ChatOptions, Context, ContextProvider
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential
from pydantic import BaseModel


class UserInfo(BaseModel):
    """사용자 정보를 저장하는 모델"""
    name: str | None = None
    age: int | None = None


class UserInfoMemory(ContextProvider):
    """사용자 정보를 기억하는 Context Provider"""
    
    def __init__(self, chat_client: ChatClientProtocol, user_info: UserInfo | None = None, **kwargs: Any):
        self._chat_client = chat_client
        if user_info:
            self.user_info = user_info
        elif kwargs:
            self.user_info = UserInfo.model_validate(kwargs)
        else:
            self.user_info = UserInfo()

    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """에이전트 호출 후 사용자 정보 추출"""
        user_messages = [
            msg for msg in request_messages 
            if hasattr(msg, "role") and msg.role.value == "user"
        ]

        if (self.user_info.name is None or self.user_info.age is None) and user_messages:
            try:
                result = await self._chat_client.get_response(
                    messages=request_messages,
                    chat_options=ChatOptions(
                        instructions="Extract the user's name and age from the message if present. If not present return nulls.",
                        response_format=UserInfo,
                    ),
                )

                if result.value and isinstance(result.value, UserInfo):
                    if result.value.name:
                        self.user_info.name = result.value.name
                    if result.value.age:
                        self.user_info.age = result.value.age

            except Exception as e:
                print(f"정보 추출 중 오류: {e}")

    async def invoking(self, messages: ChatMessage | MutableSequence[ChatMessage], **kwargs: Any) -> Context:
        """에이전트 호출 전 컨텍스트 제공"""
        instructions: list[str] = []

        if self.user_info.name is None:
            instructions.append(
                "Ask the user for their name and politely decline to answer any questions until they provide it."
            )
        else:
            instructions.append(f"The user's name is {self.user_info.name}.")

        if self.user_info.age is None:
            instructions.append(
                "Ask the user for their age and politely decline to answer any questions until they provide it."
            )
        else:
            instructions.append(f"The user's age is {self.user_info.age}.")

        return Context(instructions=" ".join(instructions))

    def serialize(self) -> str:
        """Thread 영속성을 위한 직렬화"""
        return self.user_info.model_dump_json()


print("✓ UserInfoMemory Context Provider 정의 완료")

✓ UserInfoMemory Context Provider 정의 완료


### 커스텀 Context Provider 테스트

이제 만든 Context Provider를 실제로 사용해봅시다.

In [26]:
async def test_custom_context_provider():
    """커스텀 Context Provider 테스트"""
    async with AzureCliCredential() as credential:
        chat_client = AzureAIAgentClient(async_credential=credential)
        memory_provider = UserInfoMemory(chat_client)

        async with ChatAgent(
            chat_client=chat_client,
            instructions="You are a friendly assistant. Always address the user by their name.",
            context_providers=memory_provider,
        ) as agent:
            thread = agent.get_new_thread()

            # 첫 번째 질문 - 이름과 나이를 모르는 상태
            print("[사용자] Hello, what is the square root of 9?")
            response = await agent.run("Hello, what is the square root of 9?", thread=thread)
            print(f"[에이전트] {response}\n")

            # 이름 제공
            print("[사용자] My name is 지훈")
            response = await agent.run("My name is 지훈", thread=thread)
            print(f"[에이전트] {response}\n")

            # 나이 제공
            print("[사용자] I am 25 years old")
            response = await agent.run("I am 25 years old", thread=thread)
            print(f"[에이전트] {response}\n")

            # 메모리 확인
            user_info_memory = thread.context_provider.providers[0]
            print("=== 저장된 사용자 정보 ===")
            print(f"이름: {user_info_memory.user_info.name}")
            print(f"나이: {user_info_memory.user_info.age}")

# 실행
await test_custom_context_provider()

[사용자] Hello, what is the square root of 9?


[2025-11-03 14:26:41 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[에이전트] Hello! Before I can answer your question, may I please have your name? And could you also let me know your age? I'll be happy to help once you provide these details!

[사용자] My name is 지훈


[2025-11-03 14:26:48 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/agent_framework/_clients.py:609 - WARNING] When conversation_id is set, store must be True for service-managed threads. Automatically setting store=True.


[에이전트] Thank you, 지훈! Could you please tell me your age as well? Once you share it, I'll be glad to answer your question about the square root of 9.

[사용자] I am 25 years old
[에이전트] Thank you for sharing your age, 지훈!

The square root of 9 is 3. If you have any more questions, feel free to ask!

=== 저장된 사용자 정보 ===
이름: 지훈
나이: 25
[에이전트] Thank you for sharing your age, 지훈!

The square root of 9 is 3. If you have any more questions, feel free to ask!

=== 저장된 사용자 정보 ===
이름: 지훈
나이: 25


## 3. Mem0 Context Provider

Mem0는 LLM을 위한 자가 개선 메모리 레이어로, 대화 세션 간 장기 메모리 기능을 제공합니다.

### 3.1 기본 Mem0 사용법

In [4]:
import uuid
import os

from agent_framework.mem0 import Mem0Provider
from mem0 import MemoryClient

mem_client = MemoryClient(api_key=os.getenv("MEM0_API_KEY"))

def retrieve_company_report(company_code: str, detailed: bool) -> str:
    """회사 보고서 조회 도구"""
    if company_code != "CNTS":
        raise ValueError("Company code not found")
    if not detailed:
        return "CNTS is a company that specializes in technology."
    return (
        "CNTS is a company that specializes in technology. "
        "It had a revenue of $10 million in 2022. It has 100 employees."
    )



async def test_mem0_basic():
    """Mem0 기본 사용 예제"""
    print("=== Mem0 Context Provider 기본 예제 ===")
    
    # Mem0 v2 API는 application_id와 agent_id가 필수입니다
    application_id = "workshop_app"
    agent_id = "demo_agent"
    user_id = str(uuid.uuid4())
    
    print(f"Application ID: {application_id}")
    print(f"Agent ID: {agent_id}")
    print(f"User ID: {user_id}\n")

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="FriendlyAssistant",
            instructions="You are a friendly assistant.",
            tools=retrieve_company_report,
            context_providers=Mem0Provider(
                application_id=application_id,
                agent_id=agent_id,
                user_id=user_id, 
                mem0_client=mem_client
            ),
        ) as agent,
    ):
        # 첫 번째 질문 - 회사 코드와 보고서 형식을 모르는 상태
        print("\n[사용자] Please retrieve my company report")
        result = await agent.run("Please retrieve my company report")
        print(f"[에이전트] {result}\n")

        # 회사 코드와 보고서 형식 알려주기
        query = "I always work with CNTS and I always want a detailed report format. Please remember and retrieve it."
        print(f"[사용자] {query}")
        result = await agent.run(query)
        print(f"[에이전트] {result}\n")

        print("\n=== 새로운 Thread에서 요청 ===")
        # 새 thread 생성 (이전 대화 컨텍스트 없음)
        thread = agent.get_new_thread()

        # Mem0가 사용자 선호도를 기억하므로 명확히 설명 없이도 작동
        print("[사용자] Please retrieve my company report")
        result = await agent.run("Please retrieve my company report", thread=thread)
        print(f"[에이전트] {result}\n")

# 실행 (MEM0_API_KEY가 설정되어 있어야 함)
if os.getenv("MEM0_API_KEY"):
    await test_mem0_basic()
else:
    print("⚠️  MEM0_API_KEY가 설정되지 않았습니다. Mem0 Platform을 사용하려면 API 키를 설정하세요.")

[2025-11-03 16:51:12 - /Users/andy/works/ai/maf-workshop/.venv/lib/python3.12/site-packages/mem0/client/utils.py:40 - ERROR] HTTP error occurred: Client error '400 Bad Request' for url 'https://api.mem0.ai/v2/memories/search/'
For more information check: https://developer.mozilla.org/en-US/docs/Web/HTTP/Status/400


=== Mem0 Context Provider 기본 예제 ===
Application ID: workshop_app
Agent ID: demo_agent
User ID: 85556de2-a316-4715-be4a-6c830e22200a


[사용자] Please retrieve my company report


ValidationError: {"error":"Filters are required and cannot be empty. Please refer to https://docs.mem0.ai/api-reference/memory/search-memories"}

### 3.2 Mem0 OSS (Open Source 버전)

Mem0의 오픈소스 버전을 로컬에서 실행할 수도 있습니다.

In [ ]:
# Mem0 OSS를 사용하려면 mem0ai 패키지 설치 필요
# !pip install mem0ai

from mem0 import AsyncMemory


async def test_mem0_oss():
    """Mem0 OSS 사용 예제"""
    print("=== Mem0 OSS (Open Source) 예제 ===")
    
    user_id = str(uuid.uuid4())
    
    # 로컬 Mem0 클라이언트 생성 (OpenAI API 키 필요)
    local_mem0_client = AsyncMemory(config=)
    
    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="FriendlyAssistant",
            instructions="You are a friendly assistant.",
            tools=retrieve_company_report,
            context_providers=Mem0Provider(user_id=user_id, mem0_client=local_mem0_client),
        ) as agent,
    ):
        print("\n[사용자] Please retrieve my company report")
        result = await agent.run("Please retrieve my company report")
        print(f"[에이전트] {result}\n")

        query = "I always work with CNTS and I always want a detailed report format. Please remember and retrieve it."
        print(f"[사용자] {query}")
        result = await agent.run(query)
        print(f"[에이전트] {result}\n")

        thread = agent.get_new_thread()
        print("\n[사용자] Please retrieve my company report (새 thread)")
        result = await agent.run("Please retrieve my company report", thread=thread)
        print(f"[에이전트] {result}\n")

# 실행 (OPENAI_API_KEY가 설정되어 있어야 함)
if os.getenv("LLM_AZURE_OPENAI_API_KEY"):
    try:
        await test_mem0_oss()
    except ImportError:
        print("⚠️  mem0ai 패키지가 설치되지 않았습니다. 'pip install mem0ai'로 설치하세요.")
else:
    print("⚠️  OPENAI_API_KEY가 설정되지 않았습니다. Mem0 OSS를 사용하려면 OpenAI API 키를 설정하세요.")

=== Mem0 OSS (Open Source) 예제 ===

[사용자] Please retrieve my company report


AuthenticationError: Error code: 401 - {'error': {'message': "You didn't provide an API key. You need to provide your API key in an Authorization header using Bearer auth (i.e. Authorization: Bearer YOUR_KEY), or as the password field (with blank username) if you're accessing the API from your browser and are prompted for a username and password. You can obtain an API key from https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

## 4. Redis Context Provider

Redis Context Provider는 Redis(RediSearch)를 사용하여 영구적이고 검색 가능한 메모리를 제공합니다.
전체 텍스트 검색과 벡터 임베딩을 사용한 하이브리드 검색을 지원합니다.

### 4.1 Redis 연결 설정

In [5]:
# Redis가 localhost:6379에서 실행 중이어야 합니다
# Docker로 실행: docker run --name redis -p 6379:6379 -d redis:8.0.3

from agent_framework_redis._provider import RedisProvider
from agent_framework.azure import AzureOpenAIChatClient
from redisvl.utils.vectorize import AzureOpenAITextVectorizer
from redisvl.extensions.cache.embeddings import EmbeddingsCache


def search_flights(origin_airport_code: str, destination_airport_code: str, detailed: bool = False) -> str:
    """항공편 검색 도구 (시뮬레이션)"""
    flights = {
        ("ICN", "NRT"): {
            "airline": "Korean Air",
            "duration": "2h 30m",
            "price": 250,
            "cabin": "Economy",
            "baggage": "2 checked bags",
        },
        ("ICN", "LAX"): {
            "airline": "Asiana Airlines",
            "duration": "11h 45m",
            "price": 950,
            "cabin": "Business",
            "baggage": "3 bags included",
        },
    }

    route = (origin_airport_code.upper(), destination_airport_code.upper())
    if route not in flights:
        return f"No flights found from {origin_airport_code} to {destination_airport_code}"

    flight = flights[route]
    if not detailed:
        return f"{flight['airline']} operates flights from {origin_airport_code} to {destination_airport_code}."

    return (
        f"{flight['airline']} operates flights from {origin_airport_code} to {destination_airport_code}. "
        f"Duration: {flight['duration']}. "
        f"Price: ${flight['price']}. "
        f"Cabin: {flight['cabin']}. "
        f"Baggage policy: {flight['baggage']}."
    )


print("✓ Redis 헬퍼 함수 정의 완료")

✓ Redis 헬퍼 함수 정의 완료


### 4.2 Redis Context Provider 기본 사용

In [26]:
async def test_redis_basic():
    """Redis Context Provider 기본 예제"""
    print("=== Redis Context Provider 기본 예제 ===")
    
    # OpenAI 벡터라이저 설정 (하이브리드 검색용)
    vectorizer = None
    if os.getenv("AZURE_OPENAI_API_KEY"):
        vectorizer = AzureOpenAITextVectorizer(
            model="text-embedding-ada-002",
            api_config={
                "api_key": os.getenv("AZURE_OPENAI_API_KEY"), # OR set AZURE_OPENAI
                "api_version": "2024-10-21",
                "azure_endpoint": os.getenv("AZURE_OPENAI_ENDPOINT"),
            },
            cache=EmbeddingsCache(
                name="openai_embeddings_cache",
                redis_url="redis://localhost:6379"
            ),
        )
    
    # Redis Provider 설정
    provider_config = {
        "redis_url": "redis://localhost:6379",
        "index_name": "workshop_demo",
        "prefix": "workshop_demo",
        "application_id": "workshop_app",
        "agent_id": "demo_agent",
        "user_id": "workshop_user",
        "overwrite_index": True,  # 기존 인덱스 덮어쓰기
    }
    
    if vectorizer:
        provider_config.update({
            "redis_vectorizer": vectorizer,
            "vector_field_name": "vector",
            "vector_algorithm": "hnsw",
            "vector_distance_metric": "cosine",
        })
    
    provider = RedisProvider(**provider_config)
    
    client = AzureOpenAIChatClient(
        endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
        deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
        api_version="2024-10-21",
        api_key=os.getenv("AZURE_OPENAI_API_KEY")
    )
    
    # 에이전트 생성
    agent = client.create_agent(
        name="TravelAssistant",
        instructions=(
            "You are a helpful travel assistant. "
            "Personalize replies using provided context. "
            "Before answering, always check for stored context."
        ),
        tools=[search_flights],
        context_providers=provider,
    )
    
    # 대화 진행
    print("\n[사용자] Remember that I prefer Korean Air for all my flights")
    result = await agent.run("Remember that I prefer Korean Air for all my flights")
    print(f"[에이전트] {result}\n")
    
    print("[사용자] Search for flights from ICN to LAX with full details")
    result = await agent.run("Search for flights from ICN to LAX with full details")
    print(f"[에이전트] {result}\n")
    
    # 새 thread에서 선호도 확인
    thread = agent.get_new_thread()
    print("[사용자] What airline do I prefer? (새 thread)")
    result = await agent.run("What airline do I prefer?", thread=thread)
    print(f"[에이전트] {result}\n")
    
    # 정리
    await provider.redis_index.delete()
    print("✓ Redis 인덱스 삭제 완료")

# 실행 (Redis와 OPENAI_API_KEY가 필요)
if os.getenv("AZURE_OPENAI_API_KEY"):
    try:
        await test_redis_basic()
    except Exception as e:
        print(f"⚠️  오류 발생: {e}")
        print("Redis가 localhost:6379에서 실행 중인지 확인하세요.")
else:
    print("⚠️  OPENAI_API_KEY가 설정되지 않았습니다.")

=== Redis Context Provider 기본 예제 ===

[사용자] Remember that I prefer Korean Air for all my flights
⚠️  오류 발생: Error Multiple exceptions: [Errno 61] Connect call failed ('127.0.0.1', 6379), [Errno 61] Connect call failed ('::1', 6379, 0, 0) connecting to localhost:6379.
Redis가 localhost:6379에서 실행 중인지 확인하세요.

[사용자] Remember that I prefer Korean Air for all my flights
⚠️  오류 발생: Error Multiple exceptions: [Errno 61] Connect call failed ('127.0.0.1', 6379), [Errno 61] Connect call failed ('::1', 6379, 0, 0) connecting to localhost:6379.
Redis가 localhost:6379에서 실행 중인지 확인하세요.


## 5. Thread 관리 및 스코핑

Context Provider는 다양한 스코핑 전략을 지원합니다.

### 5.1 Global Thread Scope

모든 operation에서 메모리를 공유합니다.

In [ ]:
async def test_global_thread_scope():
    """Global Thread Scope 예제"""
    print("=== Global Thread Scope 예제 ===")
    
    if not os.getenv("MEM0_API_KEY"):
        print("⚠️  MEM0_API_KEY가 필요합니다.")
        return
    
    application_id = "workshop_app"
    global_thread_id = str(uuid.uuid4())
    user_id = "user123"
    
    print(f"Application ID: {application_id}")
    print(f"User ID: {user_id}")
    print(f"Thread ID: {global_thread_id}\n")

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="GlobalMemoryAssistant",
            instructions="You are an assistant that remembers user preferences across conversations.",
            context_providers=Mem0Provider(
                application_id=application_id,
                user_id=user_id,
                thread_id=global_thread_id,
                scope_to_per_operation_thread_id=False,  # Global scope
            ),
        ) as agent,
    ):
        # Global scope에 선호도 저장
        query = "Remember that I prefer technical responses with code examples when discussing programming."
        print(f"\n[사용자] {query}")
        result = await agent.run(query)
        print(f"[에이전트] {result}\n")

        # 새 thread 생성 - global scope 덕분에 메모리 접근 가능
        new_thread = agent.get_new_thread()
        query = "What do you know about my preferences?"
        print(f"[사용자 (새 thread)] {query}")
        result = await agent.run(query, thread=new_thread)
        print(f"[에이전트] {result}\n")

await test_global_thread_scope()

=== Global Thread Scope 예제 ===
⚠️  MEM0_API_KEY가 필요합니다.


### 5.2 Per-Operation Thread Scope

각 thread마다 메모리를 격리합니다.

In [ ]:
async def test_per_operation_thread_scope():
    """Per-Operation Thread Scope 예제"""
    print("=== Per-Operation Thread Scope 예제 ===")
    
    if not os.getenv("MEM0_API_KEY"):
        print("⚠️  MEM0_API_KEY가 필요합니다.")
        return
    
    application_id = "workshop_app"
    user_id = "user123"
    
    print(f"Application ID: {application_id}")
    print(f"User ID: {user_id}\n")

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="ScopedMemoryAssistant",
            instructions="You are an assistant with thread-scoped memory.",
            context_providers=Mem0Provider(
                application_id=application_id,
                user_id=user_id,
                scope_to_per_operation_thread_id=True,  # Per-operation scope
            ),
        ) as agent,
    ):
        # 특정 thread 생성
        dedicated_thread = agent.get_new_thread()

        # 이 thread에만 정보 저장
        query = "Remember that I'm working on a Python project"
        print(f"\n[사용자] {query}")
        result = await agent.run(query, thread=dedicated_thread)
        print(f"[에이전트] {result}\n")

        # 같은 thread에서 정보 조회
        query = "What am I working on?"
        print(f"[사용자 (같은 thread)] {query}")
        result = await agent.run(query, thread=dedicated_thread)
        print(f"[에이전트] {result}\n")

        # 다른 thread에서는 정보 없음
        another_thread = agent.get_new_thread()
        query = "What am I working on?"
        print(f"[사용자 (다른 thread)] {query}")
        result = await agent.run(query, thread=another_thread)
        print(f"[에이전트] {result}\n")

await test_per_operation_thread_scope()

=== Per-Operation Thread Scope 예제 ===
⚠️  MEM0_API_KEY가 필요합니다.


### 5.3 Multiple Agents with Different Configurations

여러 에이전트가 각각 다른 메모리 설정을 가질 수 있습니다.

In [ ]:
async def test_multiple_agents():
    """여러 에이전트 메모리 격리 예제"""
    print("=== 여러 에이전트 메모리 격리 예제 ===")
    
    if not os.getenv("MEM0_API_KEY"):
        print("⚠️  MEM0_API_KEY가 필요합니다.")
        return
    
    application_id = "workshop_app"
    agent_id_1 = "agent_personal"
    agent_id_2 = "agent_work"
    user_id = "user123"
    
    print(f"Application ID: {application_id}")
    print(f"Personal Agent ID: {agent_id_1}")
    print(f"Work Agent ID: {agent_id_2}")
    print(f"User ID: {user_id}\n")

    async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="PersonalAssistant",
            instructions="You are a personal assistant that helps with personal tasks.",
            context_providers=Mem0Provider(
                application_id=application_id,
                agent_id=agent_id_1,
                user_id=user_id
            ),
        ) as personal_agent,
        AzureAIAgentClient(async_credential=credential).create_agent(
            name="WorkAssistant",
            instructions="You are a work assistant that helps with professional tasks.",
            context_providers=Mem0Provider(
                application_id=application_id,
                agent_id=agent_id_2,
                user_id=user_id
            ),
        ) as work_agent,
    ):
        # Personal agent에 정보 저장
        print("\n[사용자 -> Personal Agent] Remember I like to exercise in the morning")
        result = await personal_agent.run("Remember I like to exercise in the morning")
        print(f"[Personal Agent] {result}\n")

        # Work agent에 정보 저장
        print("[사용자 -> Work Agent] Remember I have a meeting every Monday at 9 AM")
        result = await work_agent.run("Remember I have a meeting every Monday at 9 AM")
        print(f"[Work Agent] {result}\n")

        # Personal agent는 개인 정보만 알고 있음
        print("[사용자 -> Personal Agent] What do you know about my schedule?")
        result = await personal_agent.run("What do you know about my schedule?")
        print(f"[Personal Agent] {result}\n")

        # Work agent는 업무 정보만 알고 있음
        print("[사용자 -> Work Agent] What do you know about my preferences?")
        result = await work_agent.run("What do you know about my preferences?")
        print(f"[Work Agent] {result}\n")

await test_multiple_agents()

=== 여러 에이전트 메모리 격리 예제 ===
⚠️  MEM0_API_KEY가 필요합니다.


## 요약

이 워크샵에서는 다음 내용을 학습했습니다:

1. **커스텀 Context Provider**: `ContextProvider` 인터페이스를 구현하여 자체 메모리 로직 생성
2. **Mem0 Context Provider**: Mem0 Platform 또는 OSS를 사용한 자가 개선 메모리
3. **Redis Context Provider**: Redis를 사용한 영구적이고 검색 가능한 메모리
4. **Thread 스코핑**:
   - Global scope: 모든 operation에서 메모리 공유
   - Per-operation scope: thread별 메모리 격리
   - Multiple agents: agent별 메모리 격리

### 주요 개념

- **`invoking()`**: 에이전트 호출 전에 컨텍스트 제공
- **`invoked()`**: 에이전트 호출 후에 메모리 업데이트
- **Scoping**: `application_id`, `agent_id`, `user_id`, `thread_id`를 통한 메모리 격리
- **Persistence**: 대화 세션 간 장기 메모리 유지

### 다음 단계

1. 프로덕션 환경에 맞는 Context Provider 선택
2. 적절한 스코핑 전략 결정
3. 벡터 검색 활용 (Redis, Mem0)
4. 메모리 정리 및 관리 전략 수립